In [70]:
# Import libraries section

import numpy as np
import pandas as pd
import psycopg2
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, scale
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [56]:

connection = psycopg2.connect(user='postgres',
                                    password='9853465584',
                                    host='127.0.0.1',
                                    port='5432',
                                    database='spotify')

songs = pd.read_sql_query('SELECT * FROM songs', connection, index_col='id')

users = pd.read_sql_query('SELECT * FROM users', connection, index_col='id')

connection.close()

In [57]:
print(songs.shape, users.shape)
display(songs.head())
display(users.head())

(2207, 19) (54, 3)


,song_id,song_name,artists,album,danceability,energy,song_key,loudness,song_mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_type,duration_ms,time_signature,preview_url
id,,,,,,,,,,,,,,,,,,,
1,7w87IxuO7BDcJ3YUqCyMTT,Pumped Up Kicks,Foster The People,Torches,0.733,0.71,5,-5.849,0,0.0292,0.145,0.115,0.0956,0.965,127.975,audio_features,239600,4,https://p.scdn.co/mp3-preview/db83e69f4ef8a49f...
2,5pgnhdKnYTt4MDAitK0yB1,"Symphony No. 6 in F Major, Op. 68 ""Pastoral"": ...",Ludwig van Beethoven,"Beethoven: Symphony No. 6 in F Major, Op. 68 ""...",0.322,0.0605,5,-23.267,1,0.0456,0.965,0.616,0.306,0.347,151.439,audio_features,311707,4,https://p.scdn.co/mp3-preview/d703788ab05570cd...
3,76n0kW0cXIy9Nz8AuSEAB3,Sin Ti No Sé Vivir,Los Angeles Azules,20 Kilates 20 Éxitos,0.645,0.404,2,-5.898,1,0.0373,0.316,0,0.116,0.793,84.114,audio_features,223347,4,None
4,7CUYHcu0RnbOnMz4RuN07w,Despacito (Featuring Daddy Yankee),Luis Fonsi,Despacito (Featuring Daddy Yankee),0.659,0.802,2,-4.79,1,0.148,0.208,0,0.123,0.863,177.936,audio_features,228200,4,None
5,3S4px9f4lceWdKf0gWciFu,Cheap Thrills,Sia,This Is Acting (Deluxe Version),0.628,0.698,6,-5.608,0,0.105,0.0472,0.00143,0.0907,0.732,89.976,audio_features,211667,4,https://p.scdn.co/mp3-preview/88816b2040a092aa...


,comb_id,username,song_id
id,,,
1,slinky_duck : 7w87IxuO7BDcJ3YUqCyMTT,slinky_duck,7w87IxuO7BDcJ3YUqCyMTT
2,slinky_duck : 5pgnhdKnYTt4MDAitK0yB1,slinky_duck,5pgnhdKnYTt4MDAitK0yB1
3,slinky_duck : 76n0kW0cXIy9Nz8AuSEAB3,slinky_duck,76n0kW0cXIy9Nz8AuSEAB3
4,slinky_duck : 7CUYHcu0RnbOnMz4RuN07w,slinky_duck,7CUYHcu0RnbOnMz4RuN07w
5,slinky_duck : 3S4px9f4lceWdKf0gWciFu,slinky_duck,3S4px9f4lceWdKf0gWciFu


In [59]:
encoder = LabelEncoder()

numerical_features = ['acousticness','danceability','duration_ms',
                                'energy','instrumentalness','song_key', 'liveness',
                                'loudness','song_mode','speechiness', 'tempo',
                                'time_signature','valence']

scaled_data = scale(songs[numerical_features])

encoded = encoder.fit_transform(songs['song_id'])

songs[numerical_features] = scaled_data
songs['song_id'] = encoded

songs.head()

,song_id,song_name,artists,album,danceability,energy,song_key,loudness,song_mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_type,duration_ms,time_signature,preview_url
id,,,,,,,,,,,,,,,,,,,
1,2182,Pumped Up Kicks,Foster The People,Torches,0.991894,0.456665,-0.0711403,0.547708,-1.43268,-0.450677,-0.555263,-0.108313,-0.577881,2.05142,0.310744,audio_features,-0.034349,0.247994,https://p.scdn.co/mp3-preview/db83e69f4ef8a49f...
2,1667,"Symphony No. 6 in F Major, Op. 68 ""Pastoral"": ...",Ludwig van Beethoven,"Beethoven: Symphony No. 6 in F Major, Op. 68 ""...",-1.42187,-1.96975,-0.0711403,-2.36268,0.697995,-0.343215,1.82595,1.55774,0.856749,-0.446199,1.103,audio_features,0.327867,0.247994,https://p.scdn.co/mp3-preview/d703788ab05570cd...
3,2001,Sin Ti No Sé Vivir,Los Angeles Azules,20 Kilates 20 Éxitos,0.475077,-0.686497,-0.916778,0.539521,0.697995,-0.397601,-0.0586929,-0.490739,-0.438782,1.35629,-1.17022,audio_features,-0.115993,0.247994,None
4,2036,Despacito (Featuring Daddy Yankee),Luis Fonsi,Despacito (Featuring Daddy Yankee),0.557298,0.800361,-0.916778,0.724657,0.697995,0.327765,-0.372316,-0.490739,-0.391052,1.63919,1.99767,audio_features,-0.0916147,0.247994,None
5,1034,Cheap Thrills,Sia,This Is Acting (Deluxe Version),0.375238,0.411835,0.210739,0.587977,-1.43268,0.0460055,-0.839266,-0.485984,-0.611292,1.10976,-0.972288,audio_features,-0.174665,0.247994,https://p.scdn.co/mp3-preview/88816b2040a092aa...


In [60]:
features = ['song_id','acousticness','danceability','duration_ms',
                                'energy','instrumentalness','song_key', 'liveness',
                                'loudness','song_mode','speechiness', 'tempo',
                                'time_signature','valence']
songs = songs[features]
songs.head()

,song_id,acousticness,danceability,duration_ms,energy,instrumentalness,song_key,liveness,loudness,song_mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,
1,2182,-0.555263,0.991894,-0.034349,0.456665,-0.108313,-0.0711403,-0.577881,0.547708,-1.43268,-0.450677,0.310744,0.247994,2.05142
2,1667,1.82595,-1.42187,0.327867,-1.96975,1.55774,-0.0711403,0.856749,-2.36268,0.697995,-0.343215,1.103,0.247994,-0.446199
3,2001,-0.0586929,0.475077,-0.115993,-0.686497,-0.490739,-0.916778,-0.438782,0.539521,0.697995,-0.397601,-1.17022,0.247994,1.35629
4,2036,-0.372316,0.557298,-0.0916147,0.800361,-0.490739,-0.916778,-0.391052,0.724657,0.697995,0.327765,1.99767,0.247994,1.63919
5,1034,-0.839266,0.375238,-0.174665,0.411835,-0.485984,0.210739,-0.611292,0.587977,-1.43268,0.0460055,-0.972288,0.247994,1.10976


In [64]:
X = np.array(songs.T)
X

array([[2182, 1667, 2001, ..., 2091, 1644, 601],
       [-0.555263165185306, 1.8259508371327917, -0.05869292811653199,
        ..., -0.9017007633274391, -0.19808106483759133,
        0.09231088666461565],
       [0.991893539904949, -1.4218731684782246, 0.47507731037278544, ...,
        -0.6583946475784374, 1.1680808908818232, 0.6982479549434927],
       ...,
       [0.31074350046967036, 1.1030023354944323, -1.1702173230481048,
        ..., -1.410893293728999, -0.35948910587372895,
        -0.17347845457167413],
       [0.2479943003803587, 0.2479943003803587, 0.2479943003803587, ...,
        0.2479943003803587, 0.2479943003803587, 0.2479943003803587],
       [2.051416886045506, -0.44619884999524845, 1.3562875873416071, ...,
        -0.6159397252601541, -0.04609535829939928, -0.126924346520783]],
      dtype=object)

In [71]:
def build_knn(model, outputs_size):
   
 
    # Dot product between feature vector and reference vectors
    x = Dense(units=outputs_size,
             activation='linear',
             name='dense_1',
             use_bias=False)(model.output)
    
    classifier = Model(inputs=[model.input], outputs=x)
    return classifier

In [82]:
knn_model = build_knn(Sequential, 14)
knn_model.summary()

AttributeError: type object 'Sequential' has no attribute 'ouput_shape'

In [44]:
songs.shape[1]

14